In [1]:
%run dataset.ipynb
%run models.ipynb
import time
import random
import numpy as np
from glob import glob
from tqdm import tqdm
from operator import add
from PIL import Image
import segmentation_models_pytorch as smp
import os 
import os
# os.environ["SM_FRAMEWORK"] = "tf.keras"
# import tensorflow as tf 
import re

import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
import import_ipynb
# import segmentation_models as sm

import json
from pyfiles.efficientunet import *

#!pip install pretrained-backbones-unet

from backbones_unet.model.unet import Unet
from backbones_unet.utils.dataset import SemanticSegmentationDataset
from backbones_unet.utils.trainer import Trainer
import backbones_unet
import os 
# )

# params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.AdamW(params, 1e-4) 

# trainer = Trainer(
#     model,                    # UNet model with pretrained backbone
#     criterion=DiceLoss(),     # loss function for model convergence
#     optimizer=optimizer,      # optimizer for regularization
#     epochs=10                 # number of epochs for model training
# )

# trainer.fit(train_loader, val_loader)
# print(backbones_unet.__available_models__)


In [2]:
# #loading the json file
# import json
# # Load the JSON file
# with open('dict_indices.json', 'r') as file:
#     data = json.load(file)
# train_indices = data['train_indices']
# val_indices = data['val_indices']

# # Load the JSON file
# with open('domainb_indices.json', 'r') as file:
#     data = json.load(file)

# domainb_train = data['train']
# domainb_val = data['val']

import json

import albumentations as A
from albumentations.pytorch import ToTensorV2
import re


from torch.utils.data import ConcatDataset


# Load the JSON file
with open('dict_indices.json', 'r') as file:
    data = json.load(file)
train_indices = data['train_indices']
val_indices = data['val_indices']

with open('domainb_indices.json', 'r') as file:
    data = json.load(file)

domainb_indices = data['train']


train_path = os.path.join(os.getcwd(),"train")
mask_path = os.path.join(os.getcwd(),"train_gt")
train_x = os.listdir('train')
train_x.sort(key=extract_numerical_part)
train_y = os.listdir('train_gt')
train_y.sort(key=extract_numerical_part)

val_path = os.path.join(os.getcwd(),"val")
val_gt = os.path.join(os.getcwd(),"val_gt")
val_x = os.listdir('val')
val_x.sort(key=extract_numerical_part)
val_y = os.listdir('val_gt')
val_y.sort(key=extract_numerical_part)

testB = os.path.join(os.getcwd(),"imgs")
test_gtB = os.path.join(os.getcwd(),"Gts")
path_soft_gtB  =  os.path.join(os.getcwd(),"soft_gtB")

testB_images = os.listdir(testB)
testB_images.sort(key=extract_numerical_part)
testB_masks = os.listdir(test_gtB)
testB_masks.sort(key=extract_numerical_part)



vss = dess400(train_path,mask_path,train_indices,train_transform)
vall = dess400(val_path,val_gt,val_indices,valid_transform)
dbb = dess400(testB,test_gtB,domainb_indices,train_transform)
# softb = dess400(testB,path_soft_gtB,domainb_train , train_transform)
# testb_val = dess400(testB, test_gtB ,domainb_val,train_transform)

# combined_ds_train = ConcatDataset([vss,softb ])


# train_loader = DataLoader(vss, batch_size=5, shuffle=True)
# val_loader = DataLoader(vall, batch_size=15, shuffle=False)

# mix_loader = DataLoader(combined_ds_train, batch_size=10 , shuffle=True)

vs= Dess2(train_path,mask_path,train_x ,train_y , train_transform)

val= Dess2(val_path , val_gt, val_x,val_y , valid_transform)
domainb = hotencoded(testB , test_gtB , testB_images    , testB_masks , num_classes=7,transform=valid_transform)
domainb_soft = hot400(testB , path_soft_gtB, domainb_indices, num_classes=7,transform=train_transform)
domainb_small = hot400(testB , test_gtB,domainb_indices,num_classes=7,transform=valid_transform)
ds = hot400(train_path , mask_path,train_indices,num_classes=7,transform=train_transform)
ds_val = hot400(val_path , val_gt , val_x,num_classes=7,transform=valid_transform)
# ds = hotencoded(train_path,mask_path , train_x,train_y , num_classes=7,transform=train_transform)
# ds_val = hotencoded(val_path,val_gt,val_x,val_y,num_classes=7,transform=valid_transform)


In [3]:
for x in ds_val:
    break

In [ ]:
len(val_indices), len(val_x)

In [ ]:
x[1].shape

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def train(model, loader, optimizer, loss_fn, device):
    epoch_loss = 0.0
    count = 0
    model.train()
    for x, y,_ ,_ in loader:
        x = x.to(device, dtype=torch.float32)
        y = y.float().to(device)

        optimizer.zero_grad()
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() 
        count = count + y.shape[0]
        print(count , " images processed")

    epoch_loss = epoch_loss/len(loader)
    return epoch_loss

def evaluate(model, loader, loss_fn, device):
    epoch_loss = 0.0

    model.eval()
    with torch.no_grad():
        for x, y,_ ,_ in loader:
            x = x.to(device, dtype=torch.float32)
            y = y.float().to(device)

            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            epoch_loss += loss.item()

        epoch_loss = epoch_loss / len(loader)
    return epoch_loss

In [ ]:
# model = Unet(
#     backbone='efficientnet_b4', # backbone network name
#     in_channels=3,            # input channels (1 for gray-scale images, 3 for RGB, etc.)
#     num_classes=7,            # output channels (number of classes in your dataset)
# )
# #1000 mb model size around

In [ ]:

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model = smp.Unet(
#     encoder_name="efficientnet-b4",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
#     encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
#     in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
#     classes=7,                      # model output channels (number of classes in your dataset)
# )

# model.load_state_dict(torch.load("buildunet-3.pth" , map_location=torch.device(device)))
# model.load_state_dict(torch.load("efb4-full.pth" , map_location=torch.device(device)))

In [ ]:

# img,_,mask,_=ds[0]

In [ ]:
len(ds)

In [ ]:
get_efficientunet_b4()

In [4]:
model = get_efficientunet_b4(out_channels=7, concat_input=True, pretrained=True)


In [ ]:
model = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=7,                      # model output channels (number of classes in your dataset)
)

In [3]:
old_model = smp.Unet(
    encoder_name="efficientnet-b4",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=7,                      # model output channels (number of classes in your dataset)
)
# old_model.load_state_dict(torch.load("efb4-full.pth",map_location=torch.device(device)))

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load("newefb4svd-full.pth"))

# model.load_state_dict(torch.load("newefb4svd-3200.pth" , map_location=torch.device(device)))

<All keys matched successfully>

In [6]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

checkpoint_path = "."
# Set cuda device
# model = build_unet()

# model =build_unet()

model = model.to(device)
# model.load_state_dict(torch.load("checkpoint.pth" , map_location=torch.device(device)))

# train_loader = DataLoader(vss , batch_size = 8,shuffle=True , num_workers=4)

# val_loader = DataLoader(vall , batch_size = 25,shuffle =False)
train_loader = DataLoader(ds, batch_size = 8,shuffle=True , num_workers=4)
val_loader = DataLoader(ds_val , batch_size = 20,shuffle =False)




In [7]:
# Set Optimizer and Loss

num_epochs = 100
learning_rate =0.0001

# # loss_fn = nn.CrossEntropyLoss()
loss_fn = CategoricalFocalLoss(gamma=2.0, alpha=None, reduction='mean')

# loss_fn = DiceLoss(n_classes=7,softmax=True,weight=[1,2,1,2,2,2,1])
optimizer = optim.Adam(model.parameters(), lr=learning_rate)



In [ ]:
#Train Teacher
train_losses=[]
valid_losses = []
best_valid_loss = float("inf")

In [ ]:
best_valid_loss=b_loss

In [ ]:
len(domainb)

In [ ]:

#9 done 1-4lr to 5-5 to 1-5
#softmax 2 epochs increase f1 
for epoch in range(num_epochs):
    start_time = time.time()

    train_loss = train(model, train_loader, optimizer, loss_fn, device)
    # a_loss, a_f1, a_f1_class = validlossf1(model, ds_val, loss_fn, num_classes=7)
    b_loss, b_f1, b_f1_class = validlossf1(model, ds_val, loss_fn, num_classes=7)

    train_losses.append(train_loss)
    valid_losses.append(b_loss)

    # Saving the model
    if b_loss < best_valid_loss:
        print(f"Valid loss improved from {best_valid_loss:2.4f} to {b_loss:2.4f}. Saving checkpoint: {checkpoint_path}")

        best_valid_loss = a_loss
        torch.save(model.state_dict(), "resnet34.pth")

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    data_str = f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s\n'
    data_str += f'\tTrain Loss: {train_loss:.3f}\n'
    data_str += f'\t. Loss: {b_f1:.3f}\n'
    print(data_str)

In [ ]:
# model.load_state_dict(torch.load("newefb4svd-3200.pth" , map_location=torch.device(device)))

In [8]:
old_model.load_state_dict(torch.load("efb4-full.pth"))

<All keys matched successfully>

In [9]:
old_model.to(device)
i,t,tt=validlossf1(old_model,ds_val,loss_fn,num_classes=7)
print(t)

/tmp/ipykernel_2090700/3008366362.py:25: RuntimeWarning: invalid value encountered in scalar divide
  precision = true_positives / (true_positives + false_positives)
/tmp/ipykernel_2090700/3008366362.py:26: RuntimeWarning: invalid value encountered in scalar divide
  recall = true_positives / (true_positives + false_negatives)
/tmp/ipykernel_2090700/3008366362.py:28: RuntimeWarning: invalid value encountered in scalar divide
  f1_scores[class_label] = 2 * (precision * recall) / (precision + recall)


0.86851


In [ ]:
i,t,tt=validlossf2(model,ds_val,loss_fn,num_classes=7)
print(t)
# i,t,tt=validlossf1(model2,ds_val,loss_fn,num_classes=7)
# print(t)

In [ ]:
model.load_state_dict()

In [ ]:
len(domainb_small)

In [ ]:
model.load_state_dict(torch.load("newefb4svd-full.pth" , map_location=torch.device(device)))


In [ ]:

# #restore the original mask 
# restored_mask = torch.argmax(one_hot_mask, dim=0)
# mask = mask.to(restored_mask.dtype)

# # Verify if the restored mask is exactly equal to the original mask
# is_equal = torch.allclose(mask, restored_mask)

# # Print the result
# if is_equal:
#     print("The restored mask is exactly equal to the original mask.")
# else:
#     print("The restored mask is not exactly equal to the original mask.")

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

# Define the label colors
label_colors = [
    [0, 0, 0],       # Background (black)
    [255, 0, 0],     # Class 1 (red)
    [0, 255, 0],     # Class 2 (green)
    [0, 0, 255],     # Class 3 (blue)
    [255, 255, 0],   # Class 4 (yellow)
    [255, 0, 255],   # Class 5 (magenta)
    [0, 255, 255]    # Class 6 (cyan)
]

# Iterate over the dataset
count = 0 
for image, _, mask, _ in ds_val:
    count = count + 1
    if count == 150:
        break
    old_model.eval()
    with torch.no_grad():
        result = old_model(image.unsqueeze(0).float())
        predictions = torch.argmax(result, dim=1)
        pred_labels = predictions.squeeze().cpu().numpy()

    # Create a color mask for the ground truth mask by mapping the label values to colors
    gt_color_mask = np.zeros((*mask.shape, 3), dtype=np.uint8)
    for label in range(7):  # Iterate over labels 0 to 6
        gt_color_mask[mask == label] = label_colors[label]

    # Create a color mask for the predicted mask by mapping the label values to colors
    pred_color_mask = np.zeros((*pred_labels.shape, 3), dtype=np.uint8)
    for label in range(7):  # Iterate over labels 0 to 6
        pred_color_mask[pred_labels == label] = label_colors[label]

    # Plot the input image, ground truth mask, and predicted mask
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))

    axes[0].imshow(image.permute(1, 2, 0))
    axes[0].set_title('Input Image')
    axes[0].axis('off')

    axes[1].imshow(gt_color_mask)
    axes[1].set_title('Ground Truth Mask')
    axes[1].axis('off')

    axes[2].imshow(pred_color_mask)
    axes[2].set_title('Predicted Mask')
    axes[2].axis('off')

    plt.tight_layout()
    plt.show()